# Phi-4 Mini ONNX Parallel Function Calling Tutorial

يشرح هذا الدليل كيفية استخدام Phi-4 Mini مع ONNX Runtime GenAI لاستدعاء الوظائف بشكل متوازي. يتيح استدعاء الوظائف للنموذج استخدام الأدوات الخارجية وواجهات البرمجة بناءً على طلبات المستخدم بذكاء.

## نظرة عامة

في هذا الدليل، ستتعلم كيفية:
- إعداد Phi-4 Mini مع ONNX Runtime GenAI
- تعريف مخططات الوظائف لحجز الرحلات الجوية والفنادق
- استخدام التوليد الموجه مع قواعد Lark للحصول على مخرجات منظمة
- تنفيذ استدعاءات وظائف متوازية لسيناريوهات حجز السفر المعقدة

## المتطلبات الأساسية

قبل تشغيل هذا الدليل، تأكد من:
- تنزيل نموذج Phi-4 Mini ONNX
- تثبيت حزمة `onnxruntime-genai`
- فهم أساسي لمفاهيم استدعاء الوظائف


## الخطوة 1: استيراد المكتبات المطلوبة

أولاً، سنقوم باستيراد المكتبات اللازمة لتنفيذ استدعاء الوظائف.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## الخطوة 2: إعداد النموذج وتكوينه

الآن سنقوم بتكوين نموذج Phi-4 Mini ONNX. تأكد من استبدال المسار بمجلد النموذج الخاص بك.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## الخطوة 3: إعداد معايير التوليد

قم بضبط معايير التوليد للتحكم في سلوك إخراج النموذج. تضمن هذه الإعدادات استجابات محددة ومركزة لاستدعاء الوظائف.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## الخطوة 4: تحديد الوظائف المتاحة

هنا نقوم بتحديد الوظائف التي يمكن لمساعد الذكاء الاصطناعي استخدامها. في هذا المثال، لدينا وظيفتان متعلقتان بالسفر:
1. **booking_flight_tickets**: لحجز الرحلات الجوية بين المطارات
2. **booking_hotels**: لحجز أماكن الإقامة في الفنادق

تعريف الوظائف يتبع تنسيق مخطط استدعاء الوظائف الخاص بـ OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## الخطوة 5: وظائف المساعدة لتوليد القواعد النحوية

تقوم وظائف المساعدة هذه بتحويل تعريفات وظائفنا إلى صيغة قواعد نحوية بلغة Lark، والتي تُستخدم بواسطة ONNX Runtime GenAI لتوليد موجه. يضمن هذا أن يقوم النموذج بإنتاج استدعاءات وظائف صحيحة بتنسيق JSON المناسب.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## الخطوة 6: اختبار توليد القواعد النحوية

لنقم باختبار وظائف توليد القواعد النحوية لنرى كيف تقوم بتحويل تعريفات الأدوات إلى التنسيق المناسب.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## الخطوة 7: إعداد موجه النظام والمولد

الآن سنقوم بإنشاء موجه النظام الذي يوضح للنموذج الأدوات المتاحة، وسنقوم بإعداد المولد باستخدام معايير التوليد الموجهة.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## الخطوة 8: تهيئة المولد مع التوليد الموجه

الآن سنقوم بإنشاء المولد باستخدام المعلمات التي قمنا بتكوينها وتطبيق قواعد Lark للتوليد الموجه.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## الخطوة 9: اختبار استدعاء الوظائف المتوازية

الآن دعونا نختبر إعدادنا مع سيناريو حجز سفر معقد يتطلب استدعاء عدة وظائف.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### إنشاء استدعاءات الوظائف

سيقوم النموذج الآن بإنشاء استدعاءات وظائف منظمة بناءً على طلب المستخدم. بفضل التوجيه الموجه، سيكون الإخراج بتنسيق JSON صالح يمكن تنفيذه مباشرة.

**الإخراج المتوقع**: يجب أن يقوم النموذج بإنشاء استدعاءات وظائف لـ:
1. `booking_flight_tickets` مع تفاصيل الرحلة من بكين (PEK) إلى باريس (CDG)
2. `booking_hotels` مع تفاصيل الإقامة في باريس

قم بتشغيل الخلية أدناه لرؤية التوليد المباشر:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## الخاتمة

🎉 **تهانينا!** لقد نجحت في تنفيذ استدعاء الوظائف المتوازية باستخدام Phi-4 Mini مع ONNX Runtime GenAI.

### ما تعلمته:

1. **إعداد النموذج**: كيفية إعداد Phi-4 Mini باستخدام ONNX Runtime GenAI  
2. **تعريف الوظائف**: كيفية تعريف مخططات الوظائف لاستدعاء وظائف الذكاء الاصطناعي  
3. **التوليد الموجه**: كيفية استخدام قواعد Lark للحصول على مخرجات منظمة  
4. **استدعاء الوظائف المتوازية**: كيفية التعامل مع الطلبات المعقدة التي تتطلب استدعاء وظائف متعددة  

### الفوائد الرئيسية:

- ✅ **مخرجات منظمة**: التوليد الموجه يضمن استدعاءات JSON صحيحة  
- ✅ **المعالجة المتوازية**: التعامل مع استدعاءات وظائف متعددة في طلب واحد  
- ✅ **أداء عالي**: يوفر ONNX Runtime استدلالًا محسنًا  
- ✅ **مخططات مرنة**: سهولة إضافة أو تعديل تعريفات الوظائف  

### الموارد:

- [وثائق Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)  
- [وثائق ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)  
- [أفضل الممارسات لاستدعاء الوظائف](https://platform.openai.com/docs/guides/function-calling)  



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
